# Imports and Data Read

In [157]:
import pandas as pd
import geopandas as gpd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
from sklearn.neighbors import BallTree
from tqdm import tqdm
import plotly.express as px
import re
from haversine import haversine_vector, Unit
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 100)
import seaborn as sns

import plotly.io as pio
pio.renderers.default='notebook'

In [5]:
ct_data = pd.read_csv('CT_asset_cement_emissions.csv')
ets_data = pd.read_csv('EUETS_acquiring_accounts_bytransactions.csv')

cols = ['Acquiring.Holder.MainAddressLine', 'Acquiring.Holder.SecondaryAddressLine', 'Acquiring.Holder.City','Acquiring.Holder.ZipCode','Acquiring.Holder.Country']
ets_data['Address'] = ets_data[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
ets_data['Address'] = ets_data['Address'].str.replace('nan, ','')
ct_data['st_astext'] = ct_data['st_astext'].astype(str)

ct_data[['lng', 'lat']] = ct_data['st_astext'].str.split(' ', 1, expand=True)
ct_data['lng'] = ct_data['lng'].str.replace('POINT(','',regex=False)
ct_data['lat'] = ct_data['lat'].str.replace(')','',regex=False)
ct_data['lat'] = ct_data['lat'].astype(float)
ct_data['lng'] = ct_data['lng'].astype(float)
codes = pd.read_excel('CountryCodes.xlsx')


/var/folders/h0/rfb20rk90qz0tdgv08kqcyrw0000gn/T/ipykernel_84028/1304844983.py:2: DtypeWarning: Columns (10,11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  ets_data = pd.read_csv('EUETS_acquiring_accounts_bytransactions.csv')
/var/folders/h0/rfb20rk90qz0tdgv08kqcyrw0000gn/T/ipykernel_84028/1304844983.py:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  ct_data[['lng', 'lat']] = ct_data['st_astext'].str.split(' ', 1, expand=True)


# Filtering CT data for countries that are in ETS data

In [3]:
ct_data_2digcode = pd.merge(ct_data,codes,how='left',left_on='iso3_country',right_on='Code3')
ct_data_filterETS = ct_data_2digcode[ct_data_2digcode['Code2'].isin(ets_data['Acquiring.Holder.CountryCode'].unique())]
#ct_data_filterETS.to_csv('filteredETS_CTdata.csv')

In [3]:
ct_data_filterETS = pd.read_csv('filteredETS_CTdata.csv')
ct_data_filterETS

,Unnamed: 0,asset_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,...,created_date,modified_date,asset_name,asset_type,st_astext,lng,lat,Country,Code2,Code3
0,0,1754169,BRA,cement,2016-03-01 00:00:00,2016-03-31 00:00:00,month,ch4,0,NaN,...,2022-09-05 15:22:06.555594,NaN,Cantagalo cement plant (0046),Dry,POINT(-42.271287 -21.941689),-42.271287,-21.941689,Brazil,BR,BRA
1,1,1754169,BRA,cement,2016-04-01 00:00:00,2016-04-30 00:00:00,month,ch4,0,NaN,...,2022-09-05 15:22:06.555594,NaN,Cantagalo cement plant (0046),Dry,POINT(-42.271287 -21.941689),-42.271287,-21.941689,Brazil,BR,BRA
2,2,1754169,BRA,cement,2016-05-01 00:00:00,2016-05-31 00:00:00,month,ch4,0,NaN,...,2022-09-05 15:22:06.555594,NaN,Cantagalo cement plant (0046),Dry,POINT(-42.271287 -21.941689),-42.271287,-21.941689,Brazil,BR,BRA
3,3,1754169,BRA,cement,2016-06-01 00:00:00,2016-06-30 00:00:00,month,ch4,0,NaN,...,2022-09-05 15:22:06.555594,NaN,Cantagalo cement plant (0046),Dry,POINT(-42.271287 -21.941689),-42.271287,-21.941689,Brazil,BR,BRA
4,4,1754169,BRA,cement,2016-07-01 00:00:00,2016-07-31 00:00:00,month,ch4,0,NaN,...,2022-09-05 15:22:06.555594,NaN,Cantagalo cement plant (0046),Dry,POINT(-42.271287 -21.941689),-42.271287,-21.941689,Brazil,BR,BRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923555,992195,1756298,ZAF,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2,73007,0.811607,...,2022-09-05 15:22:07.177031,NaN,Johannesburg South cement plant,NaN,POINT(28.121422 -26.223458),28.121422,-26.223458,South Africa,ZA,ZAF
923556,992196,1756298,ZAF,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,n2o,0,NaN,...,2022-09-05 15:22:07.177031,NaN,Johannesburg South cement plant,NaN,POINT(28.121422 -26.223458),28.121422,-26.223458,South Africa,ZA,ZAF
923557,992197,1756298,ZAF,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2e_100yr,73007,NaN,...,2022-09-05 15:22:07.177031,NaN,Johannesburg South cement plant,NaN,POINT(28.121422 -26.223458),28.121422,-26.223458,South Africa,ZA,ZAF
923558,992198,1756298,ZAF,cement,2015-02-01 00:00:00,2015-02-28 00:00:00,month,co2e_20yr,73007,NaN,...,2022-09-05 15:22:07.177031,NaN,Johannesburg South cement plant,NaN,POINT(28.121422 -26.223458),28.121422,-26.223458,South Africa,ZA,ZAF


# Geocoding all ETS addresses to Lat,Lng

In [7]:
locator = Nominatim(user_agent="myGeocoder")
unique_locs=ets_data['Address'].unique()
unique_locs

array(['Ziegeleistraße 14, Aschach an der Donau, 4082, Austria',
       'Bahnhofstrasse 10, Bregenz, 6900, Austria',
       'Weiberndorf 20, St. Johann i. Tirol, 6380, Austria', ...,
       'Bystrická cesta 1, Ružomberok, 3401, Slovakia',
       'Školská 470, Preseľany, 95612, Slovakia',
       'Pekná cesta 6, Bratislava, 83403, Slovakia'], dtype=object)

In [ ]:
latlngs = np.zeros((len(unique_locs),2))

for idx,loc in enumerate(tqdm(unique_locs)):
    try:
        gc = locator.geocode(loc)
        latlngs[idx,0]=gc.latitude
        latlngs[idx,1]=gc.longitude
    except:
        latlngs[idx,0]=91
        latlngs[idx,1]=91
#ets_data['point'] = ets_data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
#ets_data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [ ]:
np.savetxt("GeocodedETSLatlngs.csv", latlngs, delimiter=",")

In [4]:
latlngs = np.loadtxt('GeocodedETSLatlngs.csv',delimiter=',')
#fig = px.scatter_geo(lat=latlngs[:,0], lon=latlngs[:,1],hover_name=unique_locs)

In [8]:
latlng_df = pd.DataFrame(data=latlngs,columns=['Lat','Lng'])
latlng_df['Address'] = unique_locs
latlng_df

,Lat,Lng,Address
0,48.372008,14.016811,"Ziegeleistraße 14, Aschach an der Donau, 4082,..."
1,47.503957,9.744931,"Bahnhofstrasse 10, Bregenz, 6900, Austria"
2,47.506653,12.401352,"Weiberndorf 20, St. Johann i. Tirol, 6380, Aus..."
3,47.815973,13.051648,"Louise-Piëch-Straße 2, Salzburg, 5020, Austria"
4,48.213625,16.414014,"Trabrennstraße 6-8, Wien, 1020, Austria"
...,...,...,...
13196,91.000000,91.000000,"Námestie A. Hlinka 3/816, Žilina, 1011, Slovakia"
13197,48.290522,18.540767,"Továrenská 210, Tlmače, 93528, Slovakia"
13198,49.076710,19.315823,"Bystrická cesta 1, Ružomberok, 3401, Slovakia"
13199,91.000000,91.000000,"Školská 470, Preseľany, 95612, Slovakia"


In [9]:
ets_data_geocoded = pd.merge(ets_data,latlng_df,how='left',on='Address')
ets_data_geocoded

,TransactionID,NbOfUnits,Acquiring.AccountIDRegistryCode,Acquiring.AccountID,Acquiring.RegistryCode,Acquiring.NationalAdministrator,Acquiring.AccountStatus,Acquiring.AccountOpeningDate,Acquiring.AccountType,Acquiring.RelatedInstallationAircraftOperatorID,...,Acquiring.Holder.City,Acquiring.Holder.SecondaryAddressLine,Acquiring.Holder.RelationshipType,Acquiring.Holder.CountryCode,Acquiring.Holder.Country,Acquiring.Holder.ZipCode,Acquiring.Holder.MainAddressLine,Address,Lat,Lng
0,FR21168,2000,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
1,AT8881,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
2,AT13722,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
3,AT7617,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
4,AT17534,13646,AT10621,10621.0,AT,Austria,closed,2005-12-29 00:00:00.0,Former Operator Holding Account,NaN,...,Aschach an der Donau,Ziegeleistraße 14,Account holder,AT,Austria,4082,NaN,"Ziegeleistraße 14, Aschach an der Donau, 4082,...",48.372008,14.016811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046617,EU217344,41645,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046618,EU46951,53467,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046619,EU131484,51843,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029
1046620,EU50451,32000,XINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,46.314475,11.048029


In [10]:
ets_data_geocoded.loc[(ets_data_geocoded['Address'] == 'nan'),['Lat','Lng']] = np.nan
ets_data_geocoded.loc[(ets_data_geocoded['Lat'] == 91),['Lat','Lng']] = np.nan
ets_data_geocoded.to_csv('ETS_data_geocoded_cleaned.csv')
#Around 10% have NAN addresses

In [ ]:
ets_data_geocoded = pd.read_csv('ETS_data_geocoded_cleaned.csv')

## TODO:
* Reverse geocode all CT points
* Match CT and ETS on country AND zip code
* Generate distances between all matches pairs
* For distances below 5km

# Reverse Geocode all CT points to get Country,Zip

In [7]:
ct_addresses = np.zeros(len(ct_data.lat),dtype=str)

for idx,loc in enumerate(tqdm(ct_data.lat)):
    try:
        ct_addresses[idx]=(locator.reverse(str(ct_data.lat[idx])+','+str(ct_data.lng[idx])))
    except:
        ct_addresses[idx]='NA'
    print(ct_addresses)
    break
#ets_data['point'] = ets_data['location'].apply(lambda loc: tuple(loc.point) if loc else None)
#ets_data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

  0%|                                                                                                                                             | 0/992200 [00:00<?, ?it/s]

['N' '' '' ... '' '' '']


# Generate distance between all pairs of CT lat,lng and ETS lat,lng in same country

## TODO:
* Split CT and ETS into country blocks
* For each matching country block, calculate haversine

In [249]:
ets_country_blocks = []
ct_country_blocks = []

all_countries_ct = ct_data_filterETS['Code2'].unique()
all_countries_ets = ets_data_geocoded['Acquiring.Holder.CountryCode'].unique()

all_country_matched_plants = []

for country in all_countries_ct:
    ct_block = ct_data_filterETS[ct_data_filterETS['Code2']==country]
    ct_country_blocks.append(ct_block)
    
    ets_block = ets_data_geocoded[ets_data_geocoded['Acquiring.Holder.CountryCode']==country]
    ets_country_blocks.append(ets_block)
    
    ct_locblock = ct_block[['lat','lng']].drop_duplicates().reset_index(drop=False).rename(columns={'index':'orig_indx'})
    ets_locblock = ets_block[['Lat','Lng']].drop_duplicates().reset_index(drop=False).rename(columns={'index':'orig_indx'})
    
    dist_mat = haversine_vector(ets_locblock[['Lat','Lng']], ct_locblock[['lat','lng']], Unit.KILOMETERS, comb=True)
    
    #indices of locs <3 KM away
    close_indices = np.asarray(dist_mat<5).nonzero()
    
    matched_plants = []
    
    for idx,row in enumerate(close_indices[0]):
        ets_idx = int(ets_locblock.loc[ets_locblock.index == close_indices[1][idx]]['orig_indx'].iloc[0])
        ct_idx = int(ct_locblock.loc[ct_locblock.index == close_indices[0][idx]]['orig_indx'].iloc[0])

        matched_ets = ets_block.loc[ets_block.index == ets_idx]
        matched_ct = ct_block.loc[ct_block.index == ct_idx]
        
        matched_plants.append(pd.merge(matched_ets,matched_ct,how='inner',left_on='Acquiring.Holder.CountryCode',right_on='Code2'))
    matched_plants_df = []
    if len(matched_plants)>0:
        matched_plants_df = pd.concat(matched_plants)
    all_country_matched_plants.append(matched_plants_df)
    print('Country:',ct_block['Country'].unique(),'|No. of CT cement:',len(ct_block),'|No. of ETS:',len(ets_block),'|No. match:',len(matched_plants))

Country: ['Brazil'] |No. of CT cement: 25960 |No. of ETS: 51 |No. match: 0
Country: ['China'] |No. of CT cement: 399960 |No. of ETS: 729 |No. match: 0
Country: ['Italy'] |No. of CT cement: 15400 |No. of ETS: 46730 |No. match: 6
Country: ['India'] |No. of CT cement: 66440 |No. of ETS: 63 |No. match: 0
Country: ['Ireland'] |No. of CT cement: 1760 |No. of ETS: 5110 |No. match: 1
Country: ['Korea (the Republic of)'] |No. of CT cement: 5280 |No. of ETS: 125 |No. match: 0
Country: ['Indonesia'] |No. of CT cement: 9680 |No. of ETS: 71 |No. match: 0
Country: ['Viet Nam'] |No. of CT cement: 22000 |No. of ETS: 8 |No. match: 0
Country: ['Philippines (the)'] |No. of CT cement: 6600 |No. of ETS: 16 |No. match: 0
Country: ['France'] |No. of CT cement: 12320 |No. of ETS: 149025 |No. match: 3
Country: ['Mexico'] |No. of CT cement: 14520 |No. of ETS: 5 |No. match: 0
Country: ['Russian Federation (the)'] |No. of CT cement: 22000 |No. of ETS: 108 |No. match: 0
Country: ['United States of America (the)'] 

In [253]:
all_country_matched_plants[-3]

,TransactionID,NbOfUnits,Acquiring.AccountIDRegistryCode,Acquiring.AccountID,Acquiring.RegistryCode,Acquiring.NationalAdministrator,Acquiring.AccountStatus,Acquiring.AccountOpeningDate,Acquiring.AccountType,Acquiring.RelatedInstallationAircraftOperatorID,...,created_date,modified_date,asset_name,asset_type,st_astext,lng,lat,Country,Code2,Code3
0,EU87664,151,LU103362,103362.0,LU,Luxembourg,open,2006-06-08 00:00:00.0,Operator Holding Account,11.0,...,2022-09-05 15:22:07.021579,NaN,Kayl cement plant,Dry,POINT(6.006684 49.465827),6.006684,49.465827,Luxembourg,LU,LUX
0,EU433126,27942,LU113191,113191.0,LU,Luxembourg,open,2015-06-15 15:33:52.0,Trading Account,NaN,...,2022-09-05 15:22:07.021579,NaN,Kayl cement plant,Dry,POINT(6.006684 49.465827),6.006684,49.465827,Luxembourg,LU,LUX
0,EU454628,7279,LU116034,116034.0,LU,Luxembourg,open,2018-03-14 13:54:20.0,Operator Holding Account,210137.0,...,2022-09-05 15:22:07.021579,NaN,Kayl cement plant,Dry,POINT(6.006684 49.465827),6.006684,49.465827,Luxembourg,LU,LUX
